In [44]:
%reload_ext autoreload
%autoreload 2
from ce_builder import sensor_event_stream, watchbox, obj_group, spatialEvent, \
    ce_and, ce_until, ce_follows, event_compile
import time
import os

In [45]:
import json
data_file = open("complex_results.json", "r")
json_results = json.load(data_file)
data_file.close()

def get_data(frame_index):
    data = []
    
    for x in json_results:
        if x['time'] == frame_index:
            data.append(x)
    return data
    

In [46]:
import multiprocessing
def execute_visualization():
    os.system("python visualize.py")

In [47]:
# CE 1.1.a
#  Four recce vehicles move toward a bridge
#   What this really means is that 4 recce vehicles enters a watchbox near the bridge.

# This creates a group of rec vehicles
# rec_group_composition = {"rec_vehicle":4}
# This gets the sensor named "camera1" (in our case should refer to a video stream id)
# bridge_camera = sensor_event_stream("camera1")
bridgewatchbox1 = watchbox("camera3", positions=[213,274,772,772], id=0)
bridgewatchbox2 = watchbox("camera3", positions=[816,366,1200,725], id=1)
bridgewatchbox3 = watchbox("camera3", positions=[1294,290,1881,765], id=2)

#  Now we set up the composition requirement
# ev1 = bridgewatchbox.composition(at=0, model="rec_vehicle")==5
ev2 = bridgewatchbox1.composition(at=0, model="rec_vehicle")==4 and bridgewatchbox1.composition(at=1, model="rec_vehicle")!=4
ev2 = Event("bridgewatchbox1.composition(at=0, model="rec_vehicle")==4 and bridgewatchbox1.composition(at=1, model="rec_vehicle")!=4")
# Then we 'compile' every event, which really just creates an 'executable' event
event_to_run = event_compile(ev2)

# Run a visualization
multiprocessing.Process(target=execute_visualization).start()
time.sleep(1.5) # Visualization takes some time to set up

# Now we can iterate through some frames
for i in range(0, 60*30):
    
    # Parse our result data, and display it at the correct frame index
    incoming_data = get_data(i)
    if incoming_data:
        
        #### RUN OUR EVALUATION ON THE EVENT ANYTIME WE GET NEW DATA
        
        event_to_run.update(incoming_data)
        if event_to_run.evaluate():
            print("Event %s occurred at frame %d" %(event_to_run.event_name, i))
            break
        
        
    
    time.sleep(1/30)





bridgewatchbox1.composition(at=0, model="rec_vehicle")==4 and bridgewatchbox1.composition(at=1, model="rec_vehicle")!=4


qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 218, resource id: 0, major code: 140 (Unknown), minor code: 20


Event ev2 occurred at frame 637
